In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install deepctr[gpu]

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.2/141.2 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.4 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting unins

In [ ]:
# train
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.python.keras.models import  save_model,load_model
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from deepctr.models import MMOE,PLE

if __name__ == "__main__":
    data = pd.read_csv('./drive/MyDrive/data/course/mmoe_train_course.csv').drop(columns=['user_id','idx','course_id'])
    feature_names =data.columns.tolist()
    new_feature_list=[]
    for i in range(len(feature_names)):
        new_feature = str(i)+"_feature"
        new_feature_list.append(new_feature)
    feature_names = new_feature_list
    data.columns = feature_names
    seen_test = pd.read_csv('./drive/MyDrive/data/course/mmoe_test_seen_course.csv').drop(columns=['user_id','idx','course_id'])
    seen_test.columns = feature_names
    unseen_test = pd.read_csv('./drive/MyDrive/data/course/mmoe_test_unseen_course.csv').drop(columns=['user_id','idx','course_id'])
    unseen_test.columns = feature_names

    sparse_features = feature_names
    fixlen_feature_columns = [SparseFeat(feat, data[feat].max() + 1, embedding_dim=4) for feat in sparse_features]
    data = data.dropna()
    dnn_feature_columns = fixlen_feature_columns
    
    feature_names = sparse_features
    train_model_input = {name: data[name] for name in feature_names}
    test_seen_model_input = {name: seen_test[name] for name in feature_names}
    test_unseen_model_input = {name: unseen_test[name] for name in feature_names}
    course_info = pd.read_csv("./drive/MyDrive/data/course_list.csv")

    new_label = pd.read_csv('./drive/MyDrive/data/course/mmoe_train_course_label_multi_class.csv')
    label_name = new_label.columns.tolist()
    label = []
    for name in label_name:
        l = new_label[name].values
        label.append(l)

    model = PLE(dnn_feature_columns, task_types=['binary']*len(label_name),
                  task_names = label_name)
    model.compile("adam", loss=["binary_crossentropy"]*len(label_name),
                  metrics=['binary_crossentropy'], )
    
    history = model.fit(train_model_input, label,
                        batch_size=512, epochs=10, verbose=2,shuffle=True)
    pred_ans = model.predict(test_seen_model_input, batch_size=256)
    seen_test = pd.read_csv('./drive/MyDrive/data/course/mmoe_test_seen_course.csv',usecols=['user_id'])
    for idx,label in enumerate(label_name):
        seen_test[label] = pred_ans[idx]
    seen_test.to_csv('./drive/MyDrive/data/result/mmoe_pred_test_seen_course.csv',index=False)
    
    pred_ans = model.predict(test_unseen_model_input, batch_size=256)
    seen_test = pd.read_csv('./drive/MyDrive/data/course/mmoe_test_unseen_course.csv',usecols=['user_id'])
    for idx,label in enumerate(label_name):
        seen_test[label] = pred_ans[idx]
    seen_test.to_csv('./drive/MyDrive/data/result/mmoe_pred_test_unseen_course.csv',index=False)
    save_model(model, './drive/MyDrive/data/ckpt/PLE_course.h5')


Epoch 1/10
117/117 - 996s - loss: 58.1426 - 0_loss: 0.0677 - 1_loss: 0.0678 - 2_loss: 0.0735 - 3_loss: 0.0761 - 4_loss: 0.0696 - 5_loss: 0.0640 - 6_loss: 0.0746 - 7_loss: 0.0653 - 8_loss: 0.0678 - 9_loss: 0.0737 - 10_loss: 0.0711 - 11_loss: 0.0649 - 12_loss: 0.0771 - 13_loss: 0.0728 - 14_loss: 0.0735 - 15_loss: 0.0762 - 16_loss: 0.0686 - 17_loss: 0.0692 - 18_loss: 0.0681 - 19_loss: 0.0694 - 20_loss: 0.0674 - 21_loss: 0.0694 - 22_loss: 0.0705 - 23_loss: 0.0665 - 24_loss: 0.0702 - 25_loss: 0.0689 - 26_loss: 0.0760 - 27_loss: 0.0845 - 28_loss: 0.0750 - 29_loss: 0.0675 - 30_loss: 0.0736 - 31_loss: 0.0818 - 32_loss: 0.0800 - 33_loss: 0.0811 - 34_loss: 0.0773 - 35_loss: 0.0784 - 36_loss: 0.0647 - 37_loss: 0.0780 - 38_loss: 0.0739 - 39_loss: 0.0765 - 40_loss: 0.0779 - 41_loss: 0.0655 - 42_loss: 0.0735 - 43_loss: 0.0689 - 44_loss: 0.0731 - 45_loss: 0.0734 - 46_loss: 0.0814 - 47_loss: 0.0690 - 48_loss: 0.0712 - 49_loss: 0.0670 - 50_loss: 0.0744 - 51_loss: 0.0715 - 52_loss: 0.0750 - 53_loss: 0.1

<ipython-input-4-e56360c73fd4>:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  seen_test[label] = pred_ans[idx]
<ipython-input-4-e56360c73fd4>:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  seen_test[label] = pred_ans[idx]


In [ ]:
# test
import pandas as pd
from deepctr.layers import custom_objects
from tensorflow.python.keras.models import  save_model,load_model
import pandas as pd

from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names


seen_test = pd.read_csv('./drive/MyDrive/data/course/mmoe_test_seen_course.csv').drop(columns=['user_id','idx','course_id'])
feature_names =seen_test.columns.tolist()
new_feature_list=[]
for i in range(len(feature_names)):
    new_feature = str(i)+"_feature"
    new_feature_list.append(new_feature)
feature_names = new_feature_list
seen_test.columns = feature_names
unseen_test = pd.read_csv('./drive/MyDrive/data/course/mmoe_test_unseen_course.csv').drop(columns=['user_id','idx','course_id'])
unseen_test.columns = feature_names

sparse_features = feature_names
fixlen_feature_columns = [SparseFeat(feat, seen_test[feat].max() + 1, embedding_dim=4) for feat in sparse_features]
dnn_feature_columns = fixlen_feature_columns

feature_names = sparse_features

test_seen_model_input = {name: seen_test[name] for name in feature_names}
test_unseen_model_input = {name: unseen_test[name] for name in feature_names}



model = load_model('./drive/MyDrive/data/PLE.h5',custom_objects)# load_model,just add a parameter
pred_ans = model.predict(test_seen_model_input, batch_size=128)
seen_test = pd.read_csv('./drive/MyDrive/data/course/mmoe_test_seen_course.csv',usecols=['user_id'])
for idx,label in enumerate(label_name):
    seen_test[label] = pred_ans[idx]
seen_test.to_csv('./drive/MyDrive/data/course/mmoe_pred_test_seen_course.csv',index=False)

pred_ans = model.predict(test_unseen_model_input, batch_size=128)
unseen_test = pd.read_csv('./drive/MyDrive/data/course/mmoe_test_unseen_course.csv',usecols=['user_id'])
for idx,label in enumerate(label_name):
    unseen_test[label] = pred_ans[idx]
unseen_test.to_csv('./drive/MyDrive/data/course/mmoe_pred_test_unseen_course.csv',index=False)

<ipython-input-9-a6033a7eae57>:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  seen_test[label] = pred_ans[idx]
<ipython-input-9-a6033a7eae57>:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  seen_test[label] = pred_ans[idx]
